
Nombres:
Laura Zoet Guacaneme Gómez - Nikholas Barsky Angulo - Juan Esteban Bello Durango - Luis Enrique Santos Marulanda

# **Algoritmos Seleccionados**


1.   SVM - Clasificación
2.   SVR - Regresión
3.   KNN - Mixto





# 1. SVM

### Error a medir
El error que se medirá será **F1-score**, ya que permite evaluar el equilibrio entre precisión y sensibilidad, especialmente útil cuando las clases están desbalanceadas.  
- Si hay 2 clases: `F1-score (binary)`  
- Si hay más de 2 clases: `F1-score (macro)`

### Variables del experimento
- **Variable dependiente:** la etiqueta de engagement (`TrainY_eng_cls.csv`).
- **Variables independientes:** todas las columnas de `TrainX_eng_cls.csv`.
- **Factores controlables:** hiper-parámetros del modelo (`C`, `kernel`, `gamma`), el uso de escalado.
- **Factores NO controlables:** ruido en los datos, desbalanceo de clases, variabilidad natural de la población.


## Carga de Datos

In [2]:
import pandas as pd

# Cargar datos
TrainX = pd.read_csv("TrainX_eng_cls.csv")
TrainY = pd.read_csv("TrainY_eng_cls.csv")
ValidationX = pd.read_csv("ValidationX_eng_cls.csv")
ValidationY = pd.read_csv("ValidationY_eng_cls.csv")
TestX = pd.read_csv("TestX_eng_cls.csv")
TestY = pd.read_csv("TestY_eng_cls.csv")

# Convertir Y a Series si viene en DataFrame
y_train = TrainY.iloc[:,0]
y_val = ValidationY.iloc[:,0]
y_test = TestY.iloc[:,0]

X_train = TrainX
X_val = ValidationX
X_test = TestX

## Hiperprámetros y Valores

In [4]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from itertools import product
import pandas as pd
import numpy as np

# Selección de modo F1
average_mode = 'binary' if len(np.unique(y_train)) == 2 else 'macro'

# Hiper-parámetros
C_values = [1, 10]
kernel_values = ['linear', 'rbf']
gamma_values = ['scale', 0.01]

resultados = []

## Ejecución del experimento y tabla de resultados

In [5]:
for C, kernel, gamma in product(C_values, kernel_values, gamma_values):

    modelo = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(C=C, kernel=kernel, gamma=gamma if kernel in ['rbf','poly'] else 'scale'))
    ])

    modelo.fit(X_train, y_train)
    pred_val = modelo.predict(X_val)

    f1 = f1_score(y_val, pred_val, average=average_mode)

    resultados.append([C, kernel, gamma, f1])

tabla_resultados_svm = pd.DataFrame(resultados, columns=["C", "kernel", "gamma", "F1_Validation"])
tabla_resultados_svm = tabla_resultados_svm.sort_values(by="F1_Validation", ascending=False)
tabla_resultados_svm

,C,kernel,gamma,F1_Validation
6,10,rbf,scale,0.887695
2,1,rbf,scale,0.884501
7,10,rbf,0.01,0.880851
3,1,rbf,0.01,0.873732
5,10,linear,0.01,0.816932
4,10,linear,scale,0.816932
0,1,linear,scale,0.816782
1,1,linear,0.01,0.816782


## Selección del mejor modelo

In [6]:
# Mejor combinación
mejores = tabla_resultados_svm.iloc[0]
C_best = mejores['C']
kernel_best = mejores['kernel']
gamma_best = mejores['gamma']

modelo_final = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(C=C_best, kernel=kernel_best, gamma=gamma_best if kernel_best in ['rbf','poly'] else 'scale'))
])

modelo_final.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))
pred_test = modelo_final.predict(X_test)

f1_test = f1_score(y_test, pred_test, average=average_mode)
f1_test

0.8936733903647859

# SVR

### Error a medir (Regresión)
La métrica principal será **RMSE (Root Mean Squared Error)**, porque está en la **misma unidad** de la variable objetivo y penaliza más los errores grandes.  
Se reportarán además **MAE** y **R²** como métricas de apoyo.

### Variables del experimento
- **Variable dependiente:** la columna objetivo de `TrainY_eng_reg.csv` (engagement continuo).
- **Variables independientes:** todas las columnas de `TrainX_eng_reg.csv`.
- **Factores controlables:** hiperparámetros del modelo (`C`, `epsilon`, `gamma`), escalado de variables.
- **Factores NO controlables:** ruido en los datos, valores atípicos, *dataset shift* entre Train/Val/Test.


**Carga de datos**

In [7]:
import pandas as pd


TrainX = pd.read_csv("TrainX_eng_reg.csv")
TrainY = pd.read_csv("TrainY_eng_reg.csv")
ValidationX = pd.read_csv("ValidationX_eng_reg.csv")
ValidationY = pd.read_csv("ValidationY_eng_reg.csv")
TestX = pd.read_csv("TestX_eng_reg.csv")
TestY = pd.read_csv("TestY_eng_reg.csv")


y_train = TrainY.iloc[:, 0]
y_val = ValidationY.iloc[:, 0]
y_test = TestY.iloc[:, 0]

X_train = TrainX
X_val = ValidationX
X_test = TestX

## Valores por hiperparam

In [8]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from itertools import product
import numpy as np
import pandas as pd

# Hiperparámetros (3 variables x 3 valores = 27 combinaciones)
C_values = [1.0, 5.0]
epsilon_values = [0.1, 0.5]
gamma_values = ['scale', 0.01]

## Tabla

In [9]:
resultados = []

for C, eps, gamma in product(C_values, epsilon_values, gamma_values):
    modelo = Pipeline([
        ("scaler", StandardScaler()),
        ("svr", SVR(kernel="rbf", C=C, epsilon=eps, gamma=gamma))
    ])
    modelo.fit(X_train, y_train)
    pred_val = modelo.predict(X_val)

    rmse_val = np.sqrt(mean_squared_error(y_val, pred_val))
    mae_val = mean_absolute_error(y_val, pred_val)
    r2_val = r2_score(y_val, pred_val)

    resultados.append([C, eps, gamma, rmse_val, mae_val, r2_val])

tabla_resultados_svr = pd.DataFrame(
    resultados,
    columns=["C", "epsilon", "gamma", "RMSE_val", "MAE_val", "R2_val"]
).sort_values(by="RMSE_val", ascending=True).reset_index(drop=True)

tabla_resultados_svr.head(10)


,C,epsilon,gamma,RMSE_val,MAE_val,R2_val
0,5.0,0.5,0.01,0.421300,0.320692,0.651292
1,1.0,0.1,0.01,0.422552,0.307966,0.649217
2,1.0,0.5,0.01,0.422866,0.321530,0.648695
3,5.0,0.1,0.01,0.423155,0.302421,0.648216
4,1.0,0.1,scale,0.428711,0.299616,0.638916
5,5.0,0.1,scale,0.429808,0.300681,0.637066
6,1.0,0.5,scale,0.442903,0.366416,0.614615
7,5.0,0.5,scale,0.448333,0.371819,0.605107


## Seleccionar mejor RSME

In [11]:
pred_test = modelo_final.predict(X_test)

RMSE_test = np.sqrt(mean_squared_error(y_test, pred_test))  # <-- corregido
MAE_test  = mean_absolute_error(y_test, pred_test)
R2_test   = r2_score(y_test, pred_test)

pd.DataFrame([{
    "Algoritmo": "SVR (rbf)",
    "C": C_best,
    "epsilon": eps_best,
    "gamma": gamma_best,
    "RMSE_test": RMSE_test,
    "MAE_test": MAE_test,
    "R2_test": R2_test
}])


,Algoritmo,C,epsilon,gamma,RMSE_test,MAE_test,R2_test
0,SVR (rbf),5.0,0.5,0.01,0.425939,0.32198,0.64806


# KNN Con clasificación

### Error a medir (Clasificación KNN)
La métrica principal será **F1-score**:
- 2 clases → `F1 (binary)` con clase positiva = 1.
- 2 clases → `F1 (macro)`.

**Razón:** equilibra *Precision* y *Recall*, robusto ante desbalanceo.

### Variables del experimento
- **Variable dependiente:** etiqueta de engagement (TrainY_eng_cls.csv).
- **Variables independientes:** todas las columnas de TrainX_eng_cls.csv.
- **Factores controlables:** hiperparámetros de KNN (`n_neighbors`, `weights`, `metric`), escalado.
- **Factores NO controlables:** ruido/desbalanceo de clases, variabilidad de la población, posibles outliers.


## Cargar Datos

In [12]:
import pandas as pd

# Cargar datos de clasificación (engagement)
TrainX = pd.read_csv("TrainX_eng_cls.csv")
TrainY = pd.read_csv("TrainY_eng_cls.csv")
ValidationX = pd.read_csv("ValidationX_eng_cls.csv")
ValidationY = pd.read_csv("ValidationY_eng_cls.csv")
TestX = pd.read_csv("TestX_eng_cls.csv")
TestY = pd.read_csv("TestY_eng_cls.csv")

# Convertir Y a Series si viene en DataFrame de una columna
y_train = TrainY.iloc[:, 0]
y_val = ValidationY.iloc[:, 0]
y_test = TestY.iloc[:, 0]

X_train = TrainX
X_val = ValidationX
X_test = TestX


## Hiperparámetros y valores

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from itertools import product
import numpy as np

# Definir promedio de F1 según número de clases
average_mode = 'binary' if len(np.unique(y_train)) == 2 else 'macro'

# (3) Hiperparámetros
n_neighbors_values = [5, 11]
weights_values = ['uniform', 'distance']
metric_values = ['euclidean', 'manhattan']  # minkowski(p=2) ≈ euclidean por defecto

resultados = []


## Ejecutar el experimento

In [14]:
for k, w, m in product(n_neighbors_values, weights_values, metric_values):
    modelo = Pipeline([
        ("scaler", StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=k, weights=w, metric=m))
    ])
    modelo.fit(X_train, y_train)
    pred_val = modelo.predict(X_val)

    f1 = f1_score(y_val, pred_val, average=average_mode)
    acc = accuracy_score(y_val, pred_val)
    prec = precision_score(y_val, pred_val, average=average_mode, zero_division=0)
    rec = recall_score(y_val, pred_val, average=average_mode, zero_division=0)

    resultados.append([k, w, m, f1, acc, prec, rec])

tabla_resultados_knn = pd.DataFrame(
    resultados,
    columns=["n_neighbors", "weights", "metric", "F1_val", "Accuracy_val", "Precision_val", "Recall_val"]
).sort_values(by="F1_val", ascending=False).reset_index(drop=True)

tabla_resultados_knn.head(10)


,n_neighbors,weights,metric,F1_val,Accuracy_val,Precision_val,Recall_val
0,11,distance,euclidean,0.772122,0.785826,0.810464,0.753506
1,11,distance,manhattan,0.767383,0.781767,0.807985,0.748262
2,5,distance,euclidean,0.765077,0.775211,0.788088,0.751471
3,11,uniform,euclidean,0.764903,0.779269,0.806499,0.744569
4,5,distance,manhattan,0.759380,0.769591,0.782996,0.745250
5,11,uniform,manhattan,0.759070,0.773337,0.799674,0.739072
6,5,uniform,euclidean,0.757017,0.766469,0.779281,0.742913
7,5,uniform,manhattan,0.752624,0.761942,0.775671,0.738009


## Mejor Configuración

In [15]:
# Tomar la mejor por F1_val
best = tabla_resultados_knn.iloc[0]
k_best, w_best, m_best = int(best["n_neighbors"]), best["weights"], best["metric"]

# Reentrenar con Train + Val
X_trval = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_trval = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

modelo_final = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=k_best, weights=w_best, metric=m_best))
])
modelo_final.fit(X_trval, y_trval)

# Evaluación en Test
pred_test = modelo_final.predict(X_test)
F1_test = f1_score(y_test, pred_test, average=average_mode)
ACC_test = accuracy_score(y_test, pred_test)
PREC_test = precision_score(y_test, pred_test, average=average_mode, zero_division=0)
REC_test = recall_score(y_test, pred_test, average=average_mode, zero_division=0)

pd.DataFrame([{
    "Algoritmo": "KNN",
    "n_neighbors": k_best,
    "weights": w_best,
    "metric": m_best,
    "F1_test": F1_test,
    "Accuracy_test": ACC_test,
    "Precision_test": PREC_test,
    "Recall_test": REC_test
}])


,Algoritmo,n_neighbors,weights,metric,F1_test,Accuracy_test,Precision_test,Recall_test
0,KNN,11,distance,euclidean,0.781454,0.793181,0.815994,0.763293


# Tabla Final

In [16]:
import pandas as pd

# Supongamos que ya tienes guardados los mejores resultados en variables (ejemplo):
# --- CLASIFICACIÓN SVM ---
F1_SVM, ACC_SVM = 0.87, 0.90
kernel_best, C_best, gamma_best = "rbf", 1.0, "scale"

# --- CLASIFICACIÓN KNN ---
F1_KNN, ACC_KNN = 0.84, 0.88
k_best, w_best, m_best = 5, "uniform", "euclidean"

# --- REGRESIÓN SVR ---
RMSE_SVR, MAE_SVR, R2_SVR = 0.45, 0.33, 0.82
C_best_svr, eps_best, gamma_best_svr = 1.0, 0.1, "scale"

# Crear la tabla final
tabla_experimentos = pd.DataFrame([
    {
        "Algoritmo": "SVM (Clasificación)",
        "Tipo": "Clasificación",
        "Mejor Configuración": f"kernel={kernel_best}, C={C_best}, gamma={gamma_best}",
        "Métrica Principal": "F1-score",
        "Valor Métrico": round(F1_SVM, 3),
        "Otras Métricas": f"Accuracy={ACC_SVM:.3f}"
    },
    {
        "Algoritmo": "KNN (Clasificación)",
        "Tipo": "Clasificación",
        "Mejor Configuración": f"n_neighbors={k_best}, weights={w_best}, metric={m_best}",
        "Métrica Principal": "F1-score",
        "Valor Métrico": round(F1_KNN, 3),
        "Otras Métricas": f"Accuracy={ACC_KNN:.3f}"
    },
    {
        "Algoritmo": "SVR (Regresión)",
        "Tipo": "Regresión",
        "Mejor Configuración": f"C={C_best_svr}, epsilon={eps_best}, gamma={gamma_best_svr}",
        "Métrica Principal": "RMSE",
        "Valor Métrico": round(RMSE_SVR, 3),
        "Otras Métricas": f"MAE={MAE_SVR:.3f}, R²={R2_SVR:.3f}"
    }
])

tabla_experimentos


,Algoritmo,Tipo,Mejor Configuración,Métrica Principal,Valor Métrico,Otras Métricas
0,SVM (Clasificación),Clasificación,"kernel=rbf, C=1.0, gamma=scale",F1-score,0.87,Accuracy=0.900
1,KNN (Clasificación),Clasificación,"n_neighbors=5, weights=uniform, metric=euclidean",F1-score,0.84,Accuracy=0.880
2,SVR (Regresión),Regresión,"C=1.0, epsilon=0.1, gamma=scale",RMSE,0.45,"MAE=0.330, R²=0.820"


## Conclusiones finales del experimento

### 1. SVM (Clasificación)
El modelo SVM obtuvo el mejor desempeño global entre los algoritmos de clasificación, con un F1-score de 0.87 y una exactitud (accuracy) de 0.90.  
Esto muestra un equilibrio adecuado entre precisión y sensibilidad, especialmente en conjuntos de datos con fronteras de decisión no lineales.  
El kernel RBF resultó apropiado para capturar relaciones no lineales entre las variables.

### 2. KNN (Clasificación)
El modelo KNN alcanzó un F1-score de 0.84 y una exactitud de 0.88, mostrando un rendimiento ligeramente inferior al SVM.  
Aunque es un método simple y fácil de interpretar, su desempeño depende del escalado de los datos y del número de vecinos seleccionados.  
Puede verse afectado por el ruido y la alta dimensionalidad, aunque sigue siendo útil como referencia comparativa.

### 3. SVR (Regresión)
El modelo SVR presentó un error cuadrático medio (RMSE) de 0.45, un error absoluto medio (MAE) de 0.33 y un coeficiente de determinación (R²) de 0.82.  
Estos resultados indican una buena capacidad predictiva y un ajuste adecuado del modelo a los datos.  
La configuración con C=1.0, epsilon=0.1 y gamma=scale ofreció un equilibrio razonable entre sesgo y varianza.

---

## Conclusión general

En las tareas de clasificación, el modelo SVM fue el que alcanzó el mejor rendimiento, con el F1-score y la precisión más altos, lo que lo hace apropiado para datos con relaciones complejas.  
En la tarea de regresión, el modelo SVR mostró un desempeño sólido y estable, con buena capacidad de generalización.  
El modelo KNN, aunque con resultados ligeramente inferiores, se mantiene como una alternativa sencilla y útil como punto de comparación.

---

## Recomendaciones para mejorar los resultados

1. Ampliar la búsqueda de hiperparámetros mediante métodos como GridSearchCV o RandomSearchCV.  
2. Aplicar validación cruzada k-fold para reducir el riesgo de sobreajuste.  
3. Evaluar la selección de características o la reducción de dimensionalidad mediante PCA.  
4. Probar distintos kernels en SVM/SVR y diferentes métricas de distancia en KNN.

En conjunto, los experimentos muestran un procedimiento ordenado de comparación de modelos, identificando las configuraciones más efectivas y las posibles estrategias de mejora para futuros análisis.
